[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jburgy/blog/blob/main/linprog/allocation_ampl.ipynb)

In [ ]:
%pip install amplpy ampl-module-base ampl-module-highs --extra-index-url https://pypi.ampl.com

In [ ]:
import numpy as np
import pandas as pd
from amplpy import ampl_notebook
from google.colab import userdata

ampl = ampl_notebook(license_uuid=userdata.get("AMPL_CE_LICENSE"))

In [ ]:
%%ampl_eval
set stocks;
set accounts = {1..2};
set sectors;
# see https://groups.google.com/g/ampl/c/ULGck_3EQOM/m/yHvDLokzBQAJ
set stocks_in_sector {sectors} within stocks;

param base {accounts};
param market_value {stocks};
param sign {s in stocks} = if market_value[s] >= 0 then 1 else -1;
param volume {stocks};

var x {s in stocks, accounts}
          >= <<0; 1,0>> market_value[s],<= <<0; 0,1>> market_value[s];
var abs_x {stocks, accounts} >= 0;
var gmv >= 0;
var concentration {stocks};
var sector_mv {sectors};
var sector_nmv {sectors} >= 0;
var sector_excess {sectors};

subject to complete {s in stocks}:
          sum {a in accounts} x[s, a] = market_value[s];

subject to abs_x_def {s in stocks, a in accounts}:
          abs_x[s, a] = sign[s] * x[s, a];

subject to gmv_def:
          gmv = sum {s in stocks} abs_x[s, 1];

subject to concentration_def {s in stocks}:
          concentration[s] = abs_x[s, 1] - 0.05 * gmv;

subject to sector_mv_def {sector in sectors}:
          sector_mv[sector] = sum {s in stocks_in_sector[sector]} x[s, 1];

subject to sector_nmv_def {sector in sectors}:
          sector_nmv[sector] >= <<0; -1,1>> sector_mv[sector];

subject to sector_excess_def {sector in sectors}:
          sector_excess[sector] = sector_nmv[sector] - 0.2 * gmv;

minimize total_cost:
          sum {s in stocks, a in accounts} base[a] * abs_x[s, a]
          + sum {s in stocks} <<volume[s]; 0,0.2>> abs_x[s, 1]
          + sum {s in stocks} <<0; 0,0.1>> concentration[s]
          + sum {sector in sectors} <<0; 0,0.2>> sector_excess[sector];

In [ ]:
π = pd.read_csv(
    "https://raw.githubusercontent.com/jburgy/jupyter"
    "/refs/heads/main/content/data/portfolio.csv",
    index_col=0,
)
# Fill-in blank ticker (ESC GCI LIBERTY INC SR COMMON STOCK)
π.rename(index={np.nan: "DUMMY"}, inplace=True)
π["market_value"] = π["Shares"] * π["Price"]
π["volume"] = π["Volume"] * π["Price"]

ampl.set_data(π[["market_value", "volume"]], "stocks")
ampl.param["base"] = {1: 0.05, 2: 0.06}

stocks_in_sector = π.groupby("Sector").groups
ampl.set["sectors"] = stocks_in_sector.keys()
ampl.set["stocks_in_sector"] = stocks_in_sector

ampl.option["solver"] = "highs"
ampl.option["highs_options"] = {"outlev": 1}
ampl.solve()
assert ampl.solve_result == "solved"

In [ ]:
x = ampl.get_variable("x").get_values().to_pandas()
π.assign(
    account_1=x.loc[(π.index, 1), "x.val"].to_numpy(),
    account_2=x.loc[(π.index, 2), "x.val"].to_numpy(),
)